# PGVectorStore

```{=mdx}
:::tip 兼容性
仅适用于 Node.js。
:::
```

为了在通用 PostgreSQL 数据库中启用向量搜索，LangChain.js 支持使用 [`pgvector`](https://github.com/pgvector/pgvector) Postgres 扩展。

本指南提供了 PGVector [向量存储](/docs/concepts/#vectorstores) 的快速入门概览。如需了解所有 `PGVectorStore` 功能和配置的详细文档，请访问 [API 参考文档](https://api.js.langchain.com/classes/langchain_community_vectorstores_pgvector.PGVectorStore.html)。

## 概述

### 集成详情

| 类 | 包 | [Python 支持](https://python.langchain.com/docs/integrations/vectorstores/pgvector/) | 包的最新版本 |
| :--- | :--- | :---: | :---: |
| [`PGVectorStore`](https://api.js.langchain.com/classes/langchain_community_vectorstores_pgvector.PGVectorStore.html) | [`@langchain/community`](https://npmjs.com/@langchain/community) | ✅ | ![NPM - 版本](https://img.shields.io/npm/v/@langchain/community?style=flat-square&label=%20&) |

## 配置

要使用 PGVector 向量存储，你需要设置一个启用了 [`pgvector`](https://github.com/pgvector/pgvector) 扩展的 Postgres 实例。你还需要安装 `@langchain/community` 集成包，并将 [`pg`](https://www.npmjs.com/package/pg) 包作为对等依赖安装。

本指南还将使用 [OpenAI 嵌入](/docs/integrations/text_embedding/openai)，这要求你安装 `@langchain/openai` 集成包。如果你愿意，也可以使用 [其他支持的嵌入模型](/docs/integrations/text_embedding)。

我们还将使用 [`uuid`](https://www.npmjs.com/package/uuid) 包来生成符合要求格式的 ID。

```{=mdx}
import IntegrationInstallTooltip from "@mdx_components/integration_install_tooltip.mdx";
import Npm2Yarn from "@theme/Npm2Yarn";

<IntegrationInstallTooltip></IntegrationInstallTooltip>

<Npm2Yarn>
  @langchain/community @langchain/openai @langchain/core pg uuid
</Npm2Yarn>
```

### 设置实例

根据你设置实例的方式，有多种方式可以连接到 Postgres。以下是一个使用 `pgvector` 团队提供的预构建 Docker 镜像进行本地设置的示例。

创建一个名为 docker-compose.yml 的文件，内容如下：

```yaml
# 运行此命令以启动数据库：
# docker compose up
services:
  db:
    hostname: 127.0.0.1
    image: pgvector/pgvector:pg16
    ports:
      - 5432:5432
    restart: always
    environment:
      - POSTGRES_DB=api
      - POSTGRES_USER=myuser
      - POSTGRES_PASSWORD=ChangeMe
```

然后在同一目录中运行 `docker compose up` 来启动容器。

你可以在此 [官方仓库](https://github.com/pgvector/pgvector/) 中找到有关如何设置 pgvector 的更多信息。

### 凭据

要连接到你的 Postgres 实例，你需要相应的凭据。支持的完整选项列表，请参阅 [`node-postgres` 文档](https://node-postgres.com/apis/client)。

如果本指南中你使用的是 OpenAI 嵌入，则还需要设置你的 OpenAI 密钥：

```typescript
process.env.OPENAI_API_KEY = "YOUR_API_KEY";
```

如果你想对模型调用进行自动追踪，也可以通过取消注释以下代码来设置你的 [LangSmith](https://docs.smith.langchain.com/) API 密钥：

```typescript
// process.env.LANGSMITH_TRACING="true"
// process.env.LANGSMITH_API_KEY="your-api-key"
```

## 实例化

要实例化向量存储，请调用 `.initialize()` 静态方法。这将自动检查传入的 `config` 中指定的 `tableName` 表是否存在。如果不存在，它将使用必需的列创建该表。

```{=mdx}

::::danger 安全性
不应将用户名等用户生成的数据用作表名和列名的输入。  
**这可能导致 SQL 注入攻击！**
::::

```

In [1]:
import {
  PGVectorStore,
  DistanceStrategy,
} from "@langchain/community/vectorstores/pgvector";
import { OpenAIEmbeddings } from "@langchain/openai";
import { PoolConfig } from "pg";

const embeddings = new OpenAIEmbeddings({
  model: "text-embedding-3-small",
});

// Sample config
const config = {
  postgresConnectionOptions: {
    type: "postgres",
    host: "127.0.0.1",
    port: 5433,
    user: "myuser",
    password: "ChangeMe",
    database: "api",
  } as PoolConfig,
  tableName: "testlangchainjs",
  columns: {
    idColumnName: "id",
    vectorColumnName: "vector",
    contentColumnName: "content",
    metadataColumnName: "metadata",
  },
  // supported distance strategies: cosine (default), innerProduct, or euclidean
  distanceStrategy: "cosine" as DistanceStrategy,
};

const vectorStore = await PGVectorStore.initialize(
  embeddings,
  config
);

## 管理向量存储

### 向向量存储中添加项目

In [2]:
import { v4 as uuidv4 } from "uuid";
import type { Document } from "@langchain/core/documents";

const document1: Document = {
  pageContent: "The powerhouse of the cell is the mitochondria",
  metadata: { source: "https://example.com" }
};

const document2: Document = {
  pageContent: "Buildings are made out of brick",
  metadata: { source: "https://example.com" }
};

const document3: Document = {
  pageContent: "Mitochondria are made out of lipids",
  metadata: { source: "https://example.com" }
};

const document4: Document = {
  pageContent: "The 2024 Olympics are in Paris",
  metadata: { source: "https://example.com" }
}

const documents = [document1, document2, document3, document4];

const ids = [uuidv4(), uuidv4(), uuidv4(), uuidv4()]

await vectorStore.addDocuments(documents, { ids: ids });

### 从向量存储中删除项目

In [4]:
const id4 = ids[ids.length - 1];

await vectorStore.delete({ ids: [id4] });

## 查询向量存储

一旦创建了向量存储并添加了相关文档，您很可能希望在运行链或代理时对其进行查询。

### 直接查询

执行一个简单的相似性搜索可以按照以下方式：

In [5]:
const filter = { source: "https://example.com" };

const similaritySearchResults = await vectorStore.similaritySearch("biology", 2, filter);

for (const doc of similaritySearchResults) {
  console.log(`* ${doc.pageContent} [${JSON.stringify(doc.metadata, null)}]`);
}

* The powerhouse of the cell is the mitochondria [{"source":"https://example.com"}]
* Mitochondria are made out of lipids [{"source":"https://example.com"}]


上述过滤语法支持精确匹配，但也支持以下操作：

#### 使用 `in` 运算符

```json
{
  "field": {
    "in": ["value1", "value2"],
  }
}
```

#### 使用 `notIn` 运算符

```json
{
  "field": {
    "notIn": ["value1", "value2"],
  }
}
```

#### 使用 `arrayContains` 运算符

```json
{
  "field": {
    "arrayContains": ["value1", "value2"],
  }
}
```

如果你想执行相似性搜索并获得相应的分数，可以运行以下代码：

In [6]:
const similaritySearchWithScoreResults = await vectorStore.similaritySearchWithScore("biology", 2, filter)

for (const [doc, score] of similaritySearchWithScoreResults) {
  console.log(`* [SIM=${score.toFixed(3)}] ${doc.pageContent} [${JSON.stringify(doc.metadata)}]`);
}

* [SIM=0.835] The powerhouse of the cell is the mitochondria [{"source":"https://example.com"}]
* [SIM=0.852] Mitochondria are made out of lipids [{"source":"https://example.com"}]


### 通过转换为检索器进行查询

你还可以将向量存储转换为[检索器](/docs/concepts/retrievers)，以便在你的链中更方便地使用。

In [7]:
const retriever = vectorStore.asRetriever({
  // Optional filter
  filter: filter,
  k: 2,
});
await retriever.invoke("biology");

[
  Document {
    pageContent: 'The powerhouse of the cell is the mitochondria',
    metadata: { source: 'https://example.com' },
    id: undefined
  },
  Document {
    pageContent: 'Mitochondria are made out of lipids',
    metadata: { source: 'https://example.com' },
    id: undefined
  }
]


### 检索增强生成的用法

有关如何将此向量存储用于检索增强生成（RAG）的指南，请参阅以下部分：

- [教程：使用外部知识](/docs/tutorials/#working-with-external-knowledge)。
- [如何操作：使用 RAG 进行问答](/docs/how_to/#qa-with-rag)
- [检索概念文档](/docs/concepts/retrieval)

## 高级用法：重用连接
你可以通过创建一个连接池，然后直接通过构造函数创建新的 `PGVectorStore` 实例来重用连接。

请注意，在使用构造函数之前，你应该至少调用一次 `.initialize()` 方法，以正确设置数据库表。

In [ ]:
import { OpenAIEmbeddings } from "@langchain/openai";
import { PGVectorStore } from "@langchain/community/vectorstores/pgvector";
import pg from "pg";

// First, follow set-up instructions at
// https://js.langchain.com/docs/modules/indexes/vector_stores/integrations/pgvector

const reusablePool = new pg.Pool({
  host: "127.0.0.1",
  port: 5433,
  user: "myuser",
  password: "ChangeMe",
  database: "api",
});

const originalConfig = {
  pool: reusablePool,
  tableName: "testlangchainjs",
  collectionName: "sample",
  collectionTableName: "collections",
  columns: {
    idColumnName: "id",
    vectorColumnName: "vector",
    contentColumnName: "content",
    metadataColumnName: "metadata",
  },
};

// Set up the DB.
// Can skip this step if you've already initialized the DB.
// await PGVectorStore.initialize(new OpenAIEmbeddings(), originalConfig);
const pgvectorStore = new PGVectorStore(new OpenAIEmbeddings(), originalConfig);

await pgvectorStore.addDocuments([
  { pageContent: "what's this", metadata: { a: 2 } },
  { pageContent: "Cat drinks milk", metadata: { a: 1 } },
]);

const results = await pgvectorStore.similaritySearch("water", 1);

console.log(results);

/*
  [ Document { pageContent: 'Cat drinks milk', metadata: { a: 1 } } ]
*/

const pgvectorStore2 = new PGVectorStore(new OpenAIEmbeddings(), {
  pool: reusablePool,
  tableName: "testlangchainjs",
  collectionTableName: "collections",
  collectionName: "some_other_collection",
  columns: {
    idColumnName: "id",
    vectorColumnName: "vector",
    contentColumnName: "content",
    metadataColumnName: "metadata",
  },
});

const results2 = await pgvectorStore2.similaritySearch("water", 1);

console.log(results2);

/*
  []
*/

await reusablePool.end();

## 创建HNSW索引

默认情况下，扩展执行的是顺序扫描搜索，召回率为100%。你可以考虑为近似最近邻（ANN）搜索创建HNSW索引，以加快 `similaritySearchVectorWithScore` 的执行时间。要对你向量列创建HNSW索引，请使用 `createHnswIndex()` 方法。

方法参数包括：

- `dimensions`：定义向量数据类型的维度数量，最多2000个维度。例如，对于OpenAI的text-embedding-ada-002和Amazon的amazon.titan-embed-text-v1模型，请使用1536。

- `m?`：每层的最大连接数（默认为16）。较小的值可以改善索引构建时间，而较大的值可能会加快搜索查询。

- `efConstruction?`：用于构建图的动态候选列表的大小（默认为64）。更高的值可能会提高索引质量，但会增加索引构建时间。

- `distanceFunction?`：你想要使用的距离函数名称，会根据distanceStrategy自动选择。

更多信息请参见 [Pgvector GitHub仓库](https://github.com/pgvector/pgvector?tab=readme-ov-file#hnsw) 和 [Malkov Yu A. 与 Yashunin D. A. 于2020年发表的HNSW论文：使用分层可导航小世界图高效且鲁棒的近似最近邻搜索](https://arxiv.org/pdf/1603.09320)

In [ ]:
import { OpenAIEmbeddings } from "@langchain/openai";
import {
  DistanceStrategy,
  PGVectorStore,
} from "@langchain/community/vectorstores/pgvector";
import { PoolConfig } from "pg";

// First, follow set-up instructions at
// https://js.langchain.com/docs/modules/indexes/vector_stores/integrations/pgvector

const hnswConfig = {
  postgresConnectionOptions: {
    type: "postgres",
    host: "127.0.0.1",
    port: 5433,
    user: "myuser",
    password: "ChangeMe",
    database: "api",
  } as PoolConfig,
  tableName: "testlangchainjs",
  columns: {
    idColumnName: "id",
    vectorColumnName: "vector",
    contentColumnName: "content",
    metadataColumnName: "metadata",
  },
  // supported distance strategies: cosine (default), innerProduct, or euclidean
  distanceStrategy: "cosine" as DistanceStrategy,
};

const hnswPgVectorStore = await PGVectorStore.initialize(
  new OpenAIEmbeddings(),
  hnswConfig
);

// create the index
await hnswPgVectorStore.createHnswIndex({
  dimensions: 1536,
  efConstruction: 64,
  m: 16,
});

await hnswPgVectorStore.addDocuments([
  { pageContent: "what's this", metadata: { a: 2, b: ["tag1", "tag2"] } },
  { pageContent: "Cat drinks milk", metadata: { a: 1, b: ["tag2"] } },
]);

const model = new OpenAIEmbeddings();
const query = await model.embedQuery("water");
const hnswResults = await hnswPgVectorStore.similaritySearchVectorWithScore(query, 1);

console.log(hnswResults);

await pgvectorStore.end();

## 关闭连接

完成操作后，请确保关闭连接以避免资源过度消耗：

In [ ]:
await vectorStore.end();

## API 参考文档

如需详细了解所有 `PGVectorStore` 功能和配置，请前往 [API 参考文档](https://api.js.langchain.com/classes/langchain_community_vectorstores_pgvector.PGVectorStore.html)。